# Task 3.3 – Access Model

Import the SVM classification model from Task 3.0.3 which was trained applying a GridSearchCV.

1. Add a new endpoint `GET /accuracy` which returns the model's accuracy as a JSON-formatted object with a single attribute `accuracy`.
* Add a new endpoint `GET /best_params` which returns the modeling parameters as a JSON-formatted object with the attributes `best_params` containing `C` and `gamma` which were selected by the GridSearch as well as an attribute `param_grid` containing the parameter lists for both grid parameters. It should look like this:
```json
{
      "best_params": {
        "C": 1000.0, 
        "gamma": 0.001
      }, 
      "param_grid": {
        "C": [
          1000.0, 
          5000.0
        ], 
        "gamma": [
          0.0001, 
          0.0005
        ]
      }
}
```

In [ ]:
from flask import Flask, jsonify, request
import pandas as pd

In [ ]:
df = pd.read_csv('./data/lfw_people.csv', index_col = 0)

df['target'] = df['target'].astype('category')
target_names = df['target'].cat.categories
df['target'] = df['target'].cat.codes

X = df.iloc[:, :-1]
y = df.iloc[:, -1]

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

from sklearn.decomposition import PCA

pca = PCA(n_components=150, svd_solver='randomized', whiten=True).fit(X_train)

X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# Train a SVM classification model
param_grid = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1]}
clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced'), param_grid)
clf = clf.fit(X_train_pca, y_train)

# Best estimator found by grid search
print(clf.best_estimator_)

from sklearn.metrics import confusion_matrix

# Quantitative evaluation of the model quality on the test set
y_pred = clf.predict(X_test_pca)

print("Accuracy:", clf.score(X_test_pca, y_test))

print("Best Parameters:", clf.best_params_)

In [ ]:
app = Flask(__name__)

In [ ]:
@app.route('/accuracy')
def accuracy():
    return jsonify({'accuracy': clf.score(X_test_pca, y_test)})

In [ ]:
@app.route('/best_params')
def best_params():
    return jsonify({'best_params': clf.best_params_, 'param_grid': param_grid})

In [ ]:
app.run(debug=True, use_reloader=False, host='0.0.0.0')

*Note: To restart the app please interrupt the kernel and execute **all** cells again.*